In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from scipy.optimize import minimize 
from ft545 import myfunctions
from datetime import datetime
from scipy.optimize import fsolve

## 1.Problem 1

-Current Stock Price 165

-Strike Price 165

-Current Date 03/13/2022

-Options Expiration Date 04/15/2022

-Risk Free Rate of 0.25%

-Continuously Compounding Coupon of 0.53%

Implement the close form greeks for GBSM. Implement a finite difference derivative calculation. Compare the values between two methods for both a call and a put.

Implement the binomial tree valuation for American options with and without discrete dividends.

Assume the stock above:
    -Pays dividend on 4/11/2022 of 0.88
    
Calculate the value of the call and the put. Calculate the Greeks of each.

What is the sensitivity of the put and call to a change in the dividend amount?

In [91]:
T = datetime(2022,4,15)-datetime(2022,3,13)
T = T.days/365
S = 165
X = 165
r = 0.0025
q = 0.0053
b = r-q
sigma = 0.2

### 1.1 Implement the close form greeks

In [40]:
def d1(S,X,b,sigma,T):
    return (np.log(S/X)+(b+sigma**2/2)*T)/sigma/T**0.5
def delta_call(S,b,r,T,X,sigma):
    return np.exp((b-r)*T)*norm.cdf(d1(S,X,b,sigma,T))
def delta_put(S,b,r,T,X,sigma):
    return np.exp((b-r)*T)*(norm.cdf(d1(S,X,b,sigma,T))-1)

In [27]:
def gamma(S,b,r,T,X,sigma):
    return norm.pdf(d1(S,X,b,sigma,T))*np.exp((b-r)*T)/(S*sigma*np.sqrt(T))

In [28]:
def vega(S,b,r,T,X,sigma):
    return S*np.exp((b-r)*T)*norm.pdf(d1(S,X,b,sigma,T))*np.sqrt(T)

In [29]:
def d2(S,X,b,sigma,T):
    return d1(S,X,b,sigma,T)-sigma*T**0.5
def theta_call(S,b,r,T,X,sigma):
    return -S*np.exp((b-r)*T)*norm.pdf(d1(S,X,b,sigma,T))*sigma/2/np.sqrt(T)-(b-r)*S*np.exp((b-r)*T)*norm.cdf(d1(S,X,b,sigma,T))-r*X*np.exp(-r*T)*norm.cdf(d2(S,X,b,sigma,T))
def theta_put(S,b,r,T,X,sigma):
    return -S*np.exp((b-r)*T)*norm.pdf(d1(S,X,b,sigma,T))*sigma/2/np.sqrt(T)+(b-r)*S*np.exp((b-r)*T)*norm.cdf(-d1(S,X,b,sigma,T))+r*X*np.exp(-r*T)*norm.cdf(-d2(S,X,b,sigma,T))

In [30]:
def rho_call(S,b,r,T,X,sigma):
    return T*X*np.exp(-r*T)*norm.cdf(d2(S,X,b,sigma,T))
def rho_put(S,b,r,T,X,sigma):
    return -T*X*np.exp(-r*T)*norm.cdf(-d2(S,X,b,sigma,T))

In [31]:
def carry_rho_call(S,b,r,T,X,sigma):
    return T*S*np.exp((b-r)*T)*norm.cdf(d1(S,X,b,sigma,T))
def carry_rho_put(S,b,r,T,X,sigma):
    return -T*S*np.exp((b-r)*T)*norm.cdf(-d1(S,X,b,sigma,T))

### 1.2 Implement a finite difference derivative calculation

In [32]:
def bs_call(S,b,r,T,X,sigma):
    return S*np.exp((b-r)*T)*norm.cdf(d1(S,X,b,sigma,T))-X*np.exp(-r*T)*norm.cdf(d2(S,X,b,sigma,T))
def bs_put(S,b,r,T,X,sigma):
    return X*np.exp(-r*T)*norm.cdf(-d2(S,X,b,sigma,T))-S*np.exp((b-r)*T)*norm.cdf(-d1(S,X,b,sigma,T))

In [44]:
def fd_delta_call(S,b,r,T,X,sigma, d = 0.001*S):
    return (bs_call(S+d,b,r,T,X,sigma)-bs_call(S-d,b,r,T,X,sigma))/2/d
def fd_delta_put(S,b,r,T,X,sigma, d = 0.001*S):
    return (bs_put(S+d,b,r,T,X,sigma)-bs_put(S-d,b,r,T,X,sigma))/2/d

In [34]:
def fd_gamma(S,b,r,T,X,sigma, d = 0.001*S):
    return (bs_call(S+d,b,r,T,X,sigma)+bs_call(S-d,b,r,T,X,sigma)-2*bs_call(S,b,r,T,X,sigma))/d/d

In [35]:
def fd_vega(S,b,r,T,X,sigma, d = 0.001*sigma):
    return (bs_call(S,b,r,T,X,sigma+d)-bs_call(S,b,r,T,X,sigma-d))/2/d

In [51]:
def fd_theta_call(S,b,r,T,X,sigma, d = 0.001*T):
    return -(bs_call(S,b,r,T+d,X,sigma)-bs_call(S,b,r,T-d,X,sigma))/2/d
def fd_theta_put(S,b,r,T,X,sigma, d = 0.001*T):
    return -(bs_put(S,b,r,T+d,X,sigma)-bs_put(S,b,r,T-d,X,sigma))/2/d

In [54]:
def fd_rho_call(S,b,r,T,X,sigma, d = 0.001*r):
    return (bs_call(S,b+d,r+d,T,X,sigma)-bs_call(S,b-d,r-d,T,X,sigma))/2/d
def fd_rho_put(S,b,r,T,X,sigma, d = 0.001*r):
    return (bs_put(S,b+d,r+d,T,X,sigma)-bs_put(S,b-d,r-d,T,X,sigma))/2/d

In [55]:
def fd_carry_rho_call(S,b,r,T,X,sigma, d = 0.001*q):
    return (bs_call(S,b+d,r,T,X,sigma)-bs_call(S,b-d,r,T,X,sigma))/2/d
def fd_carry_rho_put(S,b,r,T,X,sigma, d = 0.001*q):
    return (bs_put(S,b+d,r,T,X,sigma)-bs_put(S,b-d,r,T,X,sigma))/2/d

### 1.3 Compare Values

In [57]:
print("delta call closed form vs delta call finite difference:", delta_call(S,b,r,T,X,sigma), fd_delta_call(S,b,r,T,X,sigma)) 
print("delta put closed form vs delta put finite difference:", delta_put(S,b,r,T,X,sigma), fd_delta_put(S,b,r,T,X,sigma))
print("gamma closed form vs gamma finite difference:", gamma(S,b,r,T,X,sigma), fd_gamma(S,b,r,T,X,sigma)) 
print("vega closed form vs vega finite difference:", vega(S,b,r,T,X,sigma), fd_vega(S,b,r,T,X,sigma)) 
print("theta call closed form vs theta call finite difference:", theta_call(S,b,r,T,X,sigma), fd_theta_call(S,b,r,T,X,sigma)) 
print("theta put closed form vs theta put finite difference:", theta_put(S,b,r,T,X,sigma), fd_theta_put(S,b,r,T,X,sigma))
print("rho call closed form vs rho call finite difference:", rho_call(S,b,r,T,X,sigma), fd_rho_call(S,b,r,T,X,sigma)) 
print("rho put closed form vs rho put finite difference:", rho_put(S,b,r,T,X,sigma), fd_rho_put(S,b,r,T,X,sigma))
print("carry rho call closed form vs carry rho call finite difference:", carry_rho_call(S,b,r,T,X,sigma), fd_carry_rho_call(S,b,r,T,X,sigma)) 
print("carry rho put closed form vs carry rho put finite difference:", carry_rho_put(S,b,r,T,X,sigma), fd_carry_rho_put(S,b,r,T,X,sigma))


delta call closed form vs delta call finite difference: 0.510070560620057 0.5100689809347269
delta put closed form vs delta put finite difference: -0.48945037608523323 -0.4894519557705156
gamma closed form vs gamma finite difference: 0.04017281658573558 0.0401719025425916
vega closed form vs vega finite difference: 19.776582323857255 19.776582320680802
theta call closed form vs theta call finite difference: -21.62860677878208 -21.62860951656207
theta put closed form vs theta put finite difference: -22.090281063696036 -22.09028380148204
rho call closed form vs rho call finite difference: 7.253304276901479 7.253304278265204
rho put closed form vs rho put finite difference: -7.661132489946645 -7.6611324942632555
carry rho call closed form vs carry rho call finite difference: 7.609134801578659 7.609134800484856
carry rho put closed form vs carry rho put finite difference: -7.301526843244096 -7.301526844438534


### 1.4 Implement the binomial tree valuation for American options with and without discrete dividends.

In [390]:
def bt_american_continous(is_call, S,X,T,r,b,sigma, N=200):
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(b*dt)-d)/(u-d)
    pd = 1-pu
    df = np.exp(-r*dt)
    z = is_call
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
    nNodes = nNodeFunc(N)
    optionValues = np.empty(nNodes, dtype = float)

    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i,j)
            price = S*u**i*d**(j-i)
            optionValues[idx] = max(0,z*(price-X))
            if j < N:
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)])  )
    return optionValues[0]

In [143]:
def bt_american_discrete(is_call, S,X,T,r,div_amt, div_time, sigma, N=200):
    if (len(div_amt)==0) or (len(div_time)==0):
        return bt_american_continous(is_call, S,X,T,r,r,sigma,N)
    elif div_time[0] > N:
        return bt_american_continous(is_call, S,X,T,r,r,sigma,N)

    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(b*dt)-d)/(u-d)
    pd = 1-pu
    df = np.exp(-r*dt)
    z = is_call
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
   
    nDiv = len(div_time)
    nNodes = nNodeFunc(div_time[0])

    optionValues = np.empty(nNodes, dtype = float)

    for j in range(div_time[0],-1,-1):
        for i in range(j,-1,-1):
            idx = idxFunc(i,j)
            price = S*u**i*d**(j-i)       
            
            if j < div_time[0]:
                #times before the dividend working backward induction
                optionValues[idx] = max(0,z*(price-X))
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)])  )
                
            else:
                no_ex= bt_american_discrete(is_call, price-div_amt[0], X, T-div_time[0]*dt, r, div_amt[1:nDiv], [x- div_time[0] for x in div_time[1:nDiv]], sigma, N-div_time[0])
                ex =  max(0,z*(price-X))
                optionValues[idx] = max(no_ex,ex)

    return optionValues[0]

### 1.5 Calculate the value of the call and the put. Calculate the Greeks of each. Calculate the sensitivity

In [183]:
div_amt = [0.88]
div_time = [round((datetime(2022,4,11)-datetime(2022,3,13)).days/(datetime(2022,4,15)-datetime(2022,3,13)).days*200)]
print("the value of Call with dividend is:",bt_american_discrete(1, S,X,T,r,div_amt, div_time, sigma, N=200))
print("the value of Put with dividend is:",bt_american_discrete(-1, S,X,T,r,div_amt, div_time, sigma, N=200))

the value of Call with dividend is: 3.817994047321665
the value of Put with dividend is: 4.442979957979094


In [181]:
div = 0.88
def bt_delta_call(S,b,r,T,X,sigma, d = 0.001*S):
    return (bt_american_discrete(1, S+d,X,T,r,div_amt, div_time, sigma, N=200)-bt_american_discrete(1, S-d,X,T,r,div_amt, div_time, sigma, N=200))/2/d
def bt_delta_put(S,b,r,T,X,sigma, d = 0.001*S):
    return (bt_american_discrete(-1, S+d,X,T,r,div_amt, div_time, sigma, N=200)-bt_american_discrete(-1, S-d,X,T,r,div_amt, div_time, sigma, N=200))/2/d
def bt_gamma_call(S,b,r,T,X,sigma, d = 0.001*S):
    return (bt_american_discrete(1, S+d,X,T,r,div_amt, div_time, sigma, N=200)+bt_american_discrete(1, S-d,X,T,r,div_amt, div_time, sigma, N=200)-2*bt_american_discrete(1, S,X,T,r,div_amt, div_time, sigma, N=200))/d/d
def bt_gamma_put(S,b,r,T,X,sigma, d = 0.001*S):
    return (bt_american_discrete(-1, S+d,X,T,r,div_amt, div_time, sigma, N=200)+bt_american_discrete(-1, S-d,X,T,r,div_amt, div_time, sigma, N=200)-2*bt_american_discrete(-1, S,X,T,r,div_amt, div_time, sigma, N=200))/d/d
def bt_vega_call(S,b,r,T,X,sigma, d = 0.001*sigma):
    return (bt_american_discrete(1, S,X,T,r,div_amt, div_time, sigma+d, N=200)-bt_american_discrete(1, S,X,T,r,div_amt, div_time, sigma-d, N=200))/2/d
def bt_vega_put(S,b,r,T,X,sigma, d = 0.001*sigma):
    return (bt_american_discrete(-1, S,X,T,r,div_amt, div_time, sigma+d, N=200)-bt_american_discrete(-1, S,X,T,r,div_amt, div_time, sigma-d, N=200))/2/d
def bt_theta_call(S,b,r,T,X,sigma, d = 0.001*T):
    return -(bt_american_discrete(1, S,X,T+d,r,div_amt, div_time, sigma, N=200)-bt_american_discrete(1, S,X,T-d,r,div_amt, div_time, sigma, N=200))/2/d
def bt_theta_put(S,b,r,T,X,sigma, d = 0.001*T):
    return -(bt_american_discrete(-1, S,X,T+d,r,div_amt, div_time, sigma, N=200)-bt_american_discrete(-1, S,X,T-d,r,div_amt, div_time, sigma, N=200))/2/d
def bt_rho_call(S,b,r,T,X,sigma, d = 0.001*r):
    return (bt_american_discrete(1, S,X,T,r+d,div_amt, div_time, sigma, N=200)-bt_american_discrete(1, S,X,T,r-d,div_amt, div_time, sigma, N=200))/2/d
def bt_rho_put(S,b,r,T,X,sigma, d = 0.001*r):
    return (bt_american_discrete(-1, S,X,T,r+d,div_amt, div_time, sigma, N=200)-bt_american_discrete(-1, S,X,T,r-d,div_amt, div_time, sigma, N=200))/2/d
def bt_sense_call(S,b,r,T,X,sigma, d = 0.001*div_amt[0]):
    return (bt_american_discrete(1, S,X,T,r,[div_amt[0]+d], div_time, sigma, N=200)-bt_american_discrete(1, S,X,T,r,[div_amt[0]-d], div_time, sigma, N=200))/2/d
def bt_sense_put(S,b,r,T,X,sigma, d = 0.001*div_amt[0]):
    return (bt_american_discrete(-1, S,X,T,r,[div_amt[0]+d], div_time, sigma, N=200)-bt_american_discrete(-1, S,X,T,r,[div_amt[0]-d], div_time, sigma, N=200))/2/d

In [182]:
print("delta call:", bt_delta_call(S,b,r,T,X,sigma)) 
print("delta put:", bt_delta_put(S,b,r,T,X,sigma))
print("gamma call:", bt_gamma(S,b,r,T,X,sigma)) 
print("gamma put:", bt_gamma(S,b,r,T,X,sigma)) 
print("vega call:", bt_vega(S,b,r,T,X,sigma)) 
print("vega put:", bt_vega(S,b,r,T,X,sigma))
print("theta call:", bt_theta_call(S,b,r,T,X,sigma)) 
print("theta put:", bt_theta_put(S,b,r,T,X,sigma))
print("rho call:", bt_rho_call(S,b,r,T,X,sigma)) 
print("rho put:", bt_rho_put(S,b,r,T,X,sigma))
print("sensitivity to dividend amount call", bt_sense_call(S,b,r,T,X,sigma))
print("sensitivity to dividend amount put", bt_sense_put(S,b,r,T,X,sigma))

delta call: 0.5119851469040851
delta put: -0.5173627304436692
gamma call: 0.040424651516273356
gamma put: 0.040424651516273356
vega call: 19.566465699959455
vega put: 19.566465699959455
theta call: -21.448080912971747
theta put: -22.075813821093327
rho call: -0.11056402726694613
rho put: -0.8066777397175428
sensitivity to dividend amount call -0.0918969874209989
sensitivity to dividend amount put 0.5385491011947438


## 2. Problem 2
Using the options portfolio From Problem3 last week(named problem2.csv in this week's repo) and assuming:

- American Options
- Current Date 02/25/2022
- Current AAPL price is 164.85
- Risk Free Rate of 0.25%
- Dividend Payment of 1.00 on 3/15/2022

Using DailyReturn.csv. Fit a Normal distribution to AAPL returns - assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price(above). Calculate Mean, VaR and ES.

Calculate VaR and ES using Delta-Normal.

Present all VaR and ES values as loss, not percentages.

Compare these results to last week's results.

In [233]:
S = 164.85
r = 0.0025
T = (datetime(2022,3,18)-datetime(2022,2,25)).days/365
div_amt = [1]
div_time = [round((datetime(2022,3,15)-datetime(2022,2,25)).days/(datetime(2022,3,18)-datetime(2022,2,25)).days*50)]
data = pd.read_csv("problem2.csv")
data

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice
0,Straddle,Option,AAPL,1,Call,3/18/2022,165.0,4.50
1,Straddle,Option,AAPL,1,Put,3/18/2022,165.0,4.40
2,SynLong,Option,AAPL,1,Call,3/18/2022,165.0,4.50
3,SynLong,Option,AAPL,-1,Put,3/18/2022,165.0,4.40
4,CallSpread,Option,AAPL,1,Call,3/18/2022,165.0,4.50
5,CallSpread,Option,AAPL,-1,Call,3/18/2022,175.0,0.72
6,PutSpread,Option,AAPL,1,Put,3/18/2022,165.0,4.40
7,PutSpread,Option,AAPL,-1,Put,3/18/2022,155.0,1.60
8,Stock,Stock,AAPL,1,NaN,NaN,NaN,164.85
9,Call,Option,AAPL,1,Call,3/18/2022,165.0,4.50


### 2.1 Add the implied vol using binomial tree with dividend

In [236]:
def implied_vol_call(S,b,r,T,X,price,div_amt, div_time):
    f1 = lambda z: (bt_american_discrete(1, S,X,T,r,div_amt, div_time, z, N=50)-price)
    return fsolve(f1, x0 = 0.2)[0]
def implied_vol_put(S,b,r,T,X,price, div_amt, div_time):
    f1 = lambda z: (bt_american_discrete(-1, S,X,T,r,div_amt, div_time, z, N=50)-price)
    return fsolve(f1, x0 = 0.2)[0] 
for i in range(len(data)):
    X = data.loc[i,"Strike"]
    price = data.loc[i,"CurrentPrice"]
    if price!=164.85:
        if data.loc[i,"OptionType"]=="Call":
            data.loc[i,"Implied Volatility"] = implied_vol_call(S,b,r,T,X,price, div_amt, div_time)
        else:
            data.loc[i,"Implied Volatility"] = implied_vol_put(S,b,r,T,X,price, div_amt, div_time)
    else:
        data.loc[i,"Implied Volatility"] = "NaN"
data

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,Implied Volatility
0,Straddle,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.299747
1,Straddle,Option,AAPL,1,Put,3/18/2022,165.0,4.40,0.238282
2,SynLong,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.299747
3,SynLong,Option,AAPL,-1,Put,3/18/2022,165.0,4.40,0.238282
4,CallSpread,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.299747
5,CallSpread,Option,AAPL,-1,Call,3/18/2022,175.0,0.72,0.245446
6,PutSpread,Option,AAPL,1,Put,3/18/2022,165.0,4.40,0.238282
7,PutSpread,Option,AAPL,-1,Put,3/18/2022,155.0,1.60,0.310073
8,Stock,Stock,AAPL,1,NaN,NaN,NaN,164.85,NaN
9,Call,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.299747


### 2.2 Calculate the mean, VaR and ES using Monte Carlo

In [204]:
dp = pd.read_csv("DailyReturn.csv")
data1 = list(dp.loc[:,"AAPL"])
Portfolios = ["Straddle", "SynLong", "CallSpread", "PutSpread", "Stock", "Call ", "Put ", "CoveredCall", "ProtectedPut"]
sim = 1000
cp=164.85
prices = []
for i in range(sim):
    sim_N = np.random.normal(0,np.std(data1),size=10)
    for j in sim_N:
        cp*=(1+j)
    prices.append(cp)
    cp = 164.85
Ss = prices
values = {}
for j in range(len(Ss)):
    S = Ss[j]
    for i in range(len(data)):
        name = data.loc[i,"Portfolio"]
        X = data.loc[i,"Strike"]
        sigma = data.loc[i,"Implied Volatility"]
        if data.loc[i,"Type"] == "Stock":
            value = S*data.loc[i,"Holding"]-data.loc[i,"CurrentPrice"]*data.loc[i,"Holding"]
        elif data.loc[i,"OptionType"] == "Call":
            value = bt_american_continous(1, S,X,11/365,r,r, sigma)*data.loc[i,"Holding"]-data.loc[i,"CurrentPrice"]*data.loc[i,"Holding"]
        else:
            value = bt_american_continous(-1, S,X,11/365,r,r, sigma)*data.loc[i,"Holding"]-data.loc[i,"CurrentPrice"]*data.loc[i,"Holding"]
        values[name] = values.get(name,[])
        if len(values[name])!=j+1:
            values[name].append(value)
        else:
            values[name][j]+=value
VaR = []
mean = []
ES = []
for name in Portfolios:
    arr = np.array(values[name])
    mean.append(np.mean(arr))
    VaR.append(-np.quantile(arr, 0.05))
    ES.append(-np.mean(arr[arr<np.quantile(arr, 0.05)]))
pd.DataFrame({"Mean":mean,'VaR':VaR,'ES':ES},index = Portfolios)

,Mean,VaR,ES
Straddle,0.223322,2.735163,2.748453
SynLong,0.172584,13.644135,17.350743
CallSpread,-0.193031,3.615680,3.707631
PutSpread,0.247894,2.748180,2.776602
Stock,-0.061948,13.522990,17.162870
Call,0.197953,4.335143,4.427485
Put,0.025369,4.338751,4.372083
CoveredCall,-0.259901,9.187848,12.735385
ProtectedPut,-0.036579,4.213998,4.239612


### 2.3 Add delta column

In [237]:
for i in range(len(data)):
    X = data.loc[i,"Strike"]
    price = data.loc[i,"CurrentPrice"]
    if price!=164.85:
        sigma = data.loc[i,"Implied Volatility"]
        if data.loc[i,"OptionType"]=="Call":
            data.loc[i,"delta"] = bt_delta_call(cp,b,r,T,X,sigma, d = 0.001*S)
        else:
            data.loc[i,"delta"] = bt_delta_put(cp,b,r,T,X,sigma, d = 0.001*S)
    else:
        data.loc[i,"delta"] = 1
data

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,Implied Volatility,delta
0,Straddle,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.299747,0.488306
1,Straddle,Option,AAPL,1,Put,3/18/2022,165.0,4.40,0.238282,-0.517168
2,SynLong,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.299747,0.488306
3,SynLong,Option,AAPL,-1,Put,3/18/2022,165.0,4.40,0.238282,-0.517168
4,CallSpread,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.299747,0.488306
5,CallSpread,Option,AAPL,-1,Call,3/18/2022,175.0,0.72,0.245446,0.149071
6,PutSpread,Option,AAPL,1,Put,3/18/2022,165.0,4.40,0.238282,-0.517168
7,PutSpread,Option,AAPL,-1,Put,3/18/2022,155.0,1.60,0.310073,-0.207275
8,Stock,Stock,AAPL,1,NaN,NaN,NaN,164.85,NaN,1.000000
9,Call,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.299747,0.488306


### 2.4 Calculate VaR and ES using Delta Normal

In [245]:
pvs = []
sigs = []
for i in range(len(Portfolios)):
    pi = data[data['Portfolio']==Portfolios[i]]
    pv = (pi['CurrentPrice']*pi['Holding']).sum()
    pvs.append(pv)
    sig = abs(cp/pv * (pi['Holding']*pi['delta']).sum())*np.std(data1)*np.sqrt(10)
    sigs.append(sig)
prices = []
VaR1 = []
ES1 = []
for i in range(len(Portfolios)):
    VaR1.append(-pvs[i]*norm.ppf(0.05)*sigs[i])
    ES1.append(pvs[i]*sigs[i]*norm.pdf(norm.ppf(0.05))/(0.05))
pd.DataFrame({'VaR':VaR1,'ES':ES1},index = Portfolios)

,VaR,ES
Straddle,0.393763,0.493794
SynLong,13.717957,17.202871
CallSpread,4.628281,5.804051
PutSpread,4.227954,5.302025
Stock,13.643272,17.109214
Call,6.662097,8.354539
Put,7.055860,8.848333
CoveredCall,6.981175,8.754675
ProtectedPut,6.587412,8.260881


### 2.5 Conclusion
Compared to last week's result, the values calculated by Monte Carlo are very close with minor difference in mean values. The VaR and ES calculated by delta normal have greater values in many strategies such as ProtectedPut, Call, Put. It has lower values in Straddle and CoveredCall. It's surprising to see Straddle Strategy has VaR and ES smaller than 1. 

## 3. Problem 3

Use the Fama French 3 factor return time series(F-F_Research_Data_Factors_daily.CSV) as well as the Carhart Momentum time series(F-F_momentum_Factor_daily.CSV) to fit a 4 factor model to the following stocks.

Fama stores values as percentages, you will need to divide by 100(or multiply the stock returns by 100) to get like units.

Based on the past 10 years of factor returns, find the expected annual return of each stock. 

Construct an annual covariance matrix for the 10 stocks.

Assume the risk free rate is 0.0025. Find the super efficient portfolio.

The formula is:
$$r_{s}-r_{rf} = \alpha + \beta_1*(r_{mkt}-r_{rf})+\beta_2*SMB+\beta_3*HML+\beta_4*UMD$$

In [326]:
FFR = pd.read_csv("F-F_Research_Data_Factors_daily.csv")
FFM = pd.read_csv("F-F_momentum_Factor_daily.csv")
dr = pd.read_csv('DailyReturn.csv')

In [327]:
for i in range(len(FFR)):
    FFR.loc[i,'Date'] = datetime.strptime(str(FFR.loc[i,'Date']),'%Y%m%d')
for i in range(len(FFM)):
    FFM.loc[i,'Date'] = datetime.strptime(str(FFM.loc[i,'Date']),'%Y%m%d')
for i in range(len(dr)):
    dr.loc[i,'Date'] = datetime.strptime(str(dr.loc[i,'Date']),'%m/%d/%Y')
data_FFR = FFR[FFR['Date']>= datetime.strptime("20120131",'%Y%m%d')]
data_FFM = FFM[FFM['Date']>= datetime.strptime("20120131",'%Y%m%d')]
data_FFR.set_index('Date', inplace = True)
data_FFM.set_index('Date', inplace = True)
dr.set_index('Date', inplace = True)
data_total = data_FFR.join(data_FFM)/100
dr = dr.join(data_total)

In [352]:
year_days = []
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20130131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20140131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20150131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20160131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20170131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20180131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20190131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20200131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20210131",'%Y%m%d')]))
year_days.append(len([x for x in data_total.index if x<=datetime.strptime("20220131",'%Y%m%d')]))

In [355]:
stockname= ['AAPL','FB','UNH','MA','MSFT','NVDA','HD','PFE','AMZN','BRK-B','PG','XOM','TSLA','JPM','V','DIS','GOOGL','JNJ','BAC','CSCO']
X = dr[['Mkt-RF','SMB','HML', 'Mom   ']]
X = sm.add_constant(X) 
rsss=[]
for i in stockname:
    Y = dr[i]-dr['RF']
    result = sm.OLS(Y, X).fit()
    rss = []
    annual = 1
    index = 0
    for j in range(len(data_total)):
        rs = (result.params[0]+result.params[1]*data_total.iloc[j,0]+ result.params[2]*data_total.iloc[j,1]+result.params[3]*data_total.iloc[j,2]+result.params[4]*data_total.iloc[j,4]+data_total.iloc[j,3])
        annual*=(1+rs)
        if j== year_days[index]-1:
            rss.append(annual-1)
            annual = 1
            index+=1
    print(i, np.mean(rss))
    rsss.append(rss)

AAPL 1.5454551657130924
FB 0.2333548147458683
UNH 0.5248855870608644
MA 0.13470309841474062
MSFT 0.449706709979263
NVDA 4.352215996863421
HD 0.250051481790147
PFE 2.0834606203164605
AMZN 0.10312548020738274
BRK-B 0.227364505360541
PG 0.4180087108855499
XOM 0.24836803174157182
TSLA 5.5693243322882715
JPM -0.45276930284894573
V -0.29594769275583255
DIS -0.3411675887954086
GOOGL 0.13853850809514495
JNJ -0.008845584573422337
BAC -0.2461782542730492
CSCO 0.5010356246775759


In [360]:
pd.DataFrame(np.cov(np.array(rsss)), index = stockname, columns = stockname)

,AAPL,FB,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.070909,0.020138,0.020219,-0.024050,0.052252,0.419699,0.025480,0.005241,0.020083,-0.013200,-0.005843,-0.015845,0.392962,-0.005220,-0.005318,-0.003014,0.031817,-0.005304,-0.012898,0.009936
FB,0.020138,0.036112,0.016064,0.046652,0.005924,0.028632,-0.004687,-0.129722,0.024455,0.015685,0.008025,0.026969,0.049766,0.008231,0.020668,0.017835,0.010716,0.007765,0.016601,0.019910
UNH,0.020219,0.016064,0.015158,0.021267,0.015769,0.110604,0.007311,-0.037133,0.013541,0.011758,0.010009,0.016181,0.035864,0.005397,0.008993,0.006651,0.015572,0.007399,0.009824,0.015059
MA,-0.024050,0.046652,0.021267,0.129122,-0.025606,-0.220263,-0.019030,-0.206692,0.026348,0.058830,0.036178,0.086963,-0.311556,0.027830,0.049220,0.036819,0.003055,0.028708,0.056933,0.039153
MSFT,0.052252,0.005924,0.015769,-0.025606,0.044009,0.356881,0.025297,0.049707,0.010137,-0.009377,-0.001516,-0.013875,0.273108,-0.004235,-0.008194,-0.006838,0.027046,-0.002765,-0.010806,0.006590
NVDA,0.419699,0.028632,0.110604,-0.220263,0.356881,3.046392,0.216914,0.526265,0.059237,-0.083944,-0.021351,-0.114300,2.406311,-0.035550,-0.072641,-0.065209,0.209574,-0.034445,-0.089919,0.037467
HD,0.025480,-0.004687,0.007311,-0.019030,0.025297,0.216914,0.018294,0.061364,-0.000196,-0.003704,0.001372,-0.006715,0.136169,-0.001773,-0.007121,-0.007263,0.015200,-0.001190,-0.005325,0.002123
PFE,0.005241,-0.129722,-0.037133,-0.206692,0.049707,0.526265,0.061364,0.616180,-0.079726,-0.073183,-0.032208,-0.120634,0.271660,-0.037218,-0.089543,-0.078390,0.002864,-0.033738,-0.077704,-0.065906
AMZN,0.020083,0.024455,0.013541,0.026348,0.010137,0.059237,-0.000196,-0.079726,0.018345,0.008699,0.005358,0.014341,0.053473,0.004391,0.011955,0.010739,0.011346,0.005393,0.008624,0.014368
BRK-B,-0.013200,0.015685,0.011758,0.058830,-0.009377,-0.083944,-0.003704,-0.073183,0.008699,0.032626,0.022467,0.045537,-0.182069,0.015045,0.021903,0.014781,0.004441,0.016223,0.029819,0.020190


In [389]:
def sharpe(weights):
    cov = np.cov(np.array(rsss))
    return -(np.mean(np.array(rsss), axis=1)@weights.T-0.0025)/np.sqrt(weights @ cov @ weights.T)
constraints = {'type':'eq', 'fun': lambda x: np.sum(x) - 1}
bounds = ((0.0, 1) for _ in stockname)
results = minimize(sharpe, x0=np.array([0.05]*20), constraints = constraints, bounds = bounds)
for i in range(len(results.x)):
    print(stockname[i], "has weight of", round(results.x[i]*100,2), "%")
print("The Sharpe ratio is", -results.fun)

AAPL has weight of 40.17 %
FB has weight of 0.0 %
UNH has weight of 0.0 %
MA has weight of 26.47 %
MSFT has weight of 0.0 %
NVDA has weight of 0.0 %
HD has weight of 0.0 %
PFE has weight of 14.3 %
AMZN has weight of 0.0 %
BRK-B has weight of 0.0 %
PG has weight of 19.07 %
XOM has weight of 0.0 %
TSLA has weight of 0.0 %
JPM has weight of 0.0 %
V has weight of 0.0 %
DIS has weight of 0.0 %
GOOGL has weight of 0.0 %
JNJ has weight of 0.0 %
BAC has weight of 0.0 %
CSCO has weight of 0.0 %
The Sharpe ratio is 8.553140176049073
